In [1]:
import pandas as pd
import numpy as np

In [2]:
ratings = pd.read_csv(r'C:\Users\33382\Desktop\计算机\外教\CS422\作业\homework_5\ml-100k\ml-100k\u.data', sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'])

utility_matrix = ratings.pivot_table(index='user_id', columns='item_id', values='rating')

utility_matrix.head()

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
user_means = utility_matrix.mean(axis=1)
centered_matrix = utility_matrix.sub(user_means, axis=0)

centered_matrix.head()

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.389706,-0.610294,0.389706,-0.610294,-0.610294,1.389706,0.389706,-2.610294,1.389706,-0.610294,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.290323,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.709677,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1.125714,0.125714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
user1_id = 1
user1 = centered_matrix.loc[user1_id]
similarities = []

for user in centered_matrix.index:
    if user == user1_id:
        continue
    user_centered = centered_matrix.loc[user]
    common_items = user1.notna() & user_centered.notna()
    if common_items.any():
        u1 = user1[common_items].values.astype(float)
        u = user_centered[common_items].values.astype(float)
        dot_product = np.dot(u1, u)
        norm_u1 = np.linalg.norm(u1)
        norm_u = np.linalg.norm(u)
        cosine  = dot_product / (norm_u1 * norm_u) if norm_u1 * norm_u != 0 else 0.0
    else:
        cosine  = 0.0
    similarities.append((user, cosine ))

top10_users = [user for user, cosine  in sorted(similarities, key=lambda x: x[1], reverse=True)[:10]]
print(f"\nThe 10 most similar users for user 1  are: {top10_users}\n")



The 10 most similar users for user 1  are: [418, 155, 341, 685, 812, 351, 811, 166, 810, 309]



In [5]:
item_id = 508
user1 = centered_matrix.loc[user1_id]
similarities = []

for user in centered_matrix.index:
    if user == user1_id:
        continue
    if np.isnan(utility_matrix.loc[user, item_id]):  
        continue
    user_ratings = centered_matrix.loc[user]
    common_items = user1.notna() & user_ratings.notna()
    if common_items.any():
        u1 = user1[common_items].values.astype(float)
        u = user_ratings[common_items].values.astype(float)
        dot_product = np.dot(u1, u)
        norm_u1 = np.linalg.norm(u1)
        norm_u = np.linalg.norm(u)
        cosine  = dot_product / (norm_u1 * norm_u) if norm_u1 * norm_u != 0 else 0.0
       
    else:
        cosine  = 0.0
    similarities.append((user, cosine ))

top10_users = sorted(similarities, key=lambda x: x[1], reverse=True)[:10]

numerator = 0.0
denominator = 0.0
for (user, cosine ) in top10_users:
    if cosine == 0:
        continue
    rating_centered = centered_matrix.loc[user, item_id]
    numerator += cosine  * rating_centered
    denominator += cosine 

predicted_centered = numerator / denominator if denominator != 0 else 0
predicted_rating = user_means[user1_id] + predicted_centered

print(f"\nThe predicted rating for item {item_id} by user {user1_id} is {predicted_rating:.2f}\n")


The predicted rating for item 508 by user 1 is 3.89

